In [103]:
import pandas as pd
import numpy as np
import lightgbm as lgbm
from sklearn import preprocessing
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [104]:
data = pd.read_csv("database.csv")

data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].apply(lambda x: int(str(x)[:4]))
data['Month'] = data['Date'].apply(lambda x: int(str(x)[5:7]))
data['weekday'] = data['Date'].dt.dayofweek

target = data['Magnitude']
data = data.drop(['Time', 'ID', 'Date', 'Status', 'Type', 'Magnitude'], axis=1)
train = data

#numerical
num = train.select_dtypes(include=np.number)

#categorical
cat = train.select_dtypes(exclude=np.number)
for f in cat.columns:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(cat[f].values))
    cat[f] = lbl.transform(list(cat[f].values))


train = pd.concat([cat, num], axis=1)
train = train.fillna(train.mean())

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [105]:
clf = lgbm.LGBMRegressor(boosting_type = 'gbdt')
cv = cross_validate(clf, train, target, scoring= make_scorer(mean_squared_error), cv=10)
print("Cross validation score for test set (nan values set to mean):", cv['test_score'].mean())

Cross validation score for test set (nan values set to mean): 0.1368720508602219


In [165]:
# Predicting nan values

clf = lgbm.LGBMRegressor(boosting_type = 'gbdt')

train1 = pd.concat([cat, data.select_dtypes(include=np.number)], axis=1)
nans = pd.DataFrame(train1[train1.columns[train1.isna().any()]])
for f in nans.columns:
    y = train1[f]
    x = train1.drop(f, axis=1)
    x = x.fillna(x.mean())
    not_nan = train1[f].notna()
    train_x = x.loc[not_nan, :]
    train_y = y.loc[not_nan]
    test_x = x.loc[train1[f].isna(), :]
    
    clf.fit(train_x, train_y)
    pred = clf.predict(test_x)
    train1[f][train1[f].isna()] = pred

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [168]:
cv = cross_validate(clf, train1, target, scoring= make_scorer(mean_squared_error), cv=10)
print("Cross validation score for test set (nan values are predicted):", cv['test_score'].mean())

Cross validation score for test set (nan values are predicted): 0.13753378426410445
